# Create Image - GAF e MTF
---
Questo notebook trasforma serie temporali in immagini tramite tecniche GAF e MTF.


## Import libraries

In [3]:
import pandas as pd
import numpy as np
from pyts.image import GramianAngularField, MarkovTransitionField
import matplotlib.pyplot as plt
import os

## Load Data

In [4]:
# path
path = "/media/neurone-pc6/Volume/Michele/Prog_GAF_Michele/pythonProject/data/Vasiceksimulation_windows_23_07.csv"csv"

# load
df = pd.read_csv(path)

## Transform Time Series into Images

In [3]:
# Functions 
def compute_gaf(data: np.ndarray, method: str) -> np.ndarray:
    """
    Compute the Gramian Angular Field (GAF) of the input data using the specified method.

    Parameters:
    - data (np.ndarray): A 2D array of numerical values where each row represents a time series.
    - method (str): The method to use for GAF calculation; options are typically 'summation' or 'difference'.

    Returns:
    - np.ndarray: A transformed array representing the GAF, with the same shape as the input data.
    """
    gaf = GramianAngularField(image_size=data.shape[1], method=method)
    transformed = gaf.transform(data)
    return transformed.squeeze()

def compute_markov(data: np.ndarray) -> np.ndarray:
    """
    Compute the Markov Transition Field (MTF) for the input data.

    Parameters:
    - data (np.ndarray): A 2D array of numerical values where each row represents a time series.

    Returns:
    - np.ndarray: A transformed array representing the MTF, with the same shape as the input data.
    """
    mtf = MarkovTransitionField(image_size=data.shape[1])
    transformed = mtf.fit_transform(data)
    return transformed.squeeze()

def GAF_MARKOV_RGB(data: pd.DataFrame) -> np.ndarray:
    """
    Transform input data into an RGB image representing the Gramian Angular Field (GAF),
    using the Markov Transition Field (MTF) as the third channel.

    Parameters:
    - data (pd.DataFrame): A DataFrame where each row represents a time series.

    Returns:
    - np.ndarray: An RGB image where the three channels correspond to the GASF, GADF, and MTF respectively.
    """
    # Create GramianAngularField instances for GASF and GADF calculation
    gasf = GramianAngularField(image_size=data.shape[1], method='summation')
    gadf = GramianAngularField(image_size=data.shape[1], method='difference')
    
    # Transform the data using GASF and GADF
    gasf_transformed = gasf.transform(data)
    gadf_transformed = gadf.transform(data)

    # Compute the Markov Transition Field
    mtf_transformed = compute_markov(data)
    
    # Combine GASF, GADF, and MTF results into an RGB image
    gaf_transformed = np.stack((gasf_transformed.squeeze(), gadf_transformed.squeeze(), mtf_transformed), axis=-1)
    
    # Normalize the values to the [0, 255] range and convert to integers
    gaf_transformed = (gaf_transformed / np.max(gaf_transformed) * 255).astype('uint8')
    
    return gaf_transformed

def save_image(image: np.ndarray, path: str):
    """
    Save the image to a file in .npy format.

    Parameters:
    - image (np.ndarray): The image data to be saved.
    - path (str): The file path where the image will be saved.
    """
    np.save(path, image)


In [5]:
# Save path
base_path = "/media/neurone-pc6/Volume/Michele/Prog_GAF_Michele/pythonProject/data"

# folders
methods = ['GADF', 'GASF', 'MARKOV', 'RGB']
labels = ['Label_0', 'Label_1']

for method in methods:
    for label in labels:
        directory = os.path.join(base_path, method, label)
        if not os.path.exists(directory):
            os.makedirs(directory)

In [6]:
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    series = row[:-1].to_numpy().reshape(1, -1)  # Extract the series, excluding the label column
    label = row[-1]  # Extract the label of the series

    # Convert the label to a string
    label_str = f'Label_{int(label)}'

    # Calculate the images for each method
    gaf_image = compute_gaf(series, 'difference')  # GADF
    gasf_image = compute_gaf(series, 'summation')  # GASF
    markov_image = compute_markov(series)  # MARKOV
    rgb_image = GAF_MARKOV_RGB(series)  # RGB

    # Save the images
    save_image(gaf_image, os.path.join(base_path, 'GADF', label_str, f'{index}.npy'))
    save_image(gasf_image, os.path.join(base_path, 'GASF', label_str, f'{index}.npy'))
    save_image(markov_image, os.path.join(base_path, 'MARKOV', label_str, f'{index}.npy'))
    save_image(rgb_image, os.path.join(base_path, 'RGB', label_str, f'{index}.npy'))